In [1]:
#1.Library
import pandas as pd
from openpyxl import load_workbook
import time
import numpy as np
#1.1.Record the timer
start_time = time.time()


In [2]:
#2.Creating a DataFrame from archive Itens_Sem_Pertence

#2.1 Read Itens_Sem_Pertence File
df = pd.read_excel('Itens_Sem_Pertence .xlsx', sheet_name='base', engine = 'openpyxl')

#2.2 Defining parameters from df
df = df.loc[(df['DAT_DIA_CALEND'] == '2021-11-02') & (df['COD_LOCAL'] == 1965)]
#2.2.1 Formating Date
formated_df = df['DAT_DIA_CALEND'].dt.strftime("%d/%m/%Y")

#2.3 Formating new column
df.drop('DAT_DIA_CALEND',axis='columns', inplace=True)
#2.3.1 Insert new column with date formating
df.insert(0,"DAT_DIA_CALEND",formated_df)



#2.4 Read HA8 File
df_HA8 = pd.read_csv('HA8.txt', delimiter = ";",skipinitialspace=True, skiprows=1,low_memory=False)
#2.4.1 Select with columns with index
df_HA8 = df_HA8.iloc[:, [19,20,23,45,62,63,64,65]]
#2.4.2 Fixed columns name
df_HA8 = df_HA8.rename(columns={'PLU    ': 'PLU', 'ESTOQUE   ': 'ESTOQUE','TRANSF        ':'TRANSF'})
#2.4.3 Formating DF
df_HA8['ESTOQUE'] = df_HA8['ESTOQUE'].str.replace(",0", "")
df_HA8['TRANSF'] = df_HA8['TRANSF'].str.replace(",", ".")
df_HA8['BLOQ.ARMAZ.EXT'] = df_HA8['BLOQ.ARMAZ.EXT'].str.replace(",0", "")
df_HA8['QTD.ARMAZ.EXT'] = df_HA8['QTD.ARMAZ.EXT'].str.replace(",0", "")
df_HA8['QTD.BLOQ.GQ'] = df_HA8['QTD.BLOQ.GQ'].str.replace(",0", "")
df_HA8['QTD.BLOQ.ADM'] = df_HA8['QTD.BLOQ.ADM'].str.replace(",0", "")
df_HA8[['PLU','ESTOQUE','EMB TR','BLOQ.ARMAZ.EXT','QTD.ARMAZ.EXT','QTD.BLOQ.GQ','QTD.BLOQ.ADM']] = df_HA8[['PLU','ESTOQUE','EMB TR','BLOQ.ARMAZ.EXT','QTD.ARMAZ.EXT','QTD.BLOQ.GQ','QTD.BLOQ.ADM']].fillna(0).astype(np.int64)
df_HA8['TRANSF'] = df_HA8['TRANSF'].fillna(0).astype(np.float64)
df_HA8.insert(0,"BLOQ",None)
df_HA8.insert(1,"TESTOQUE",None)
df_HA8.insert(1,"CUST",None)

#2.5 Read H1J File
df_H1J = pd.read_csv('H1J.txt', delimiter = ";", skiprows=1,low_memory=False,thousands=',')
df_H1J = df_H1J[['P L U  ']]
df_H1J.insert(1,"POSIÇÕES",df_H1J)
df_H1J = df_H1J.groupby('P L U  ')['POSIÇÕES'].count().reset_index()

In [3]:
#3 Union tables
df = pd.merge(df,df_H1J,left_on=['COD_PLU'], right_on=['P L U  '], how='left')
df.drop('P L U  ',axis='columns', inplace=True)
df_HA8['BLOQ'] = df_HA8['BLOQ.ARMAZ.EXT'] + df_HA8['QTD.ARMAZ.EXT'] + df_HA8['QTD.BLOQ.GQ'] + df_HA8['QTD.BLOQ.ADM']
df_HA8['TESTOQUE'] = df_HA8['ESTOQUE'] * df_HA8['EMB TR']
df_HA8['CUST'] = df_HA8['TESTOQUE'] * df_HA8['TRANSF']
df = pd.merge(df,df_HA8[['BLOQ','TESTOQUE','CUST','PLU','ESTOQUE','EMB TR']],left_on=['COD_PLU'], right_on=['PLU'], how='left')
df.drop('PLU',axis='columns', inplace=True)

In [4]:
#4 Formation DF
df = df.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,28,29,26,27,18,19,20,21,22,23,24,25]]
df = df.rename(columns={'ESTOQUE': 'Estoque Caixas', 'EMB TR':'Embalagem','TESTOQUE':'CD_ESTOQUE_UND','CUST':'CD_ESTOQUE_R$'})
df = df.loc[df['BLOQ'] == 0]
df.drop('BLOQ',axis='columns', inplace=True)

In [5]:
#5 Download to excel
df.to_excel("Itens_1965.xlsx", sheet_name='base',index=False)

In [6]:
#6 Finish
print("This script executed in %s seconds " % (time.time() - start_time))

This script executed in 34.76969575881958 seconds 
